In [13]:
from azureml.core.run import Run
from azureml.core.model import Model
import mlflow
import os
from warnings import warn
from time import time
import numpy as np
from matplotlib import pyplot as plt
try:
    from sklearnex import patch_sklearn
    patch_sklearn() 
except:
    warn('unable to import sklearn intel ex module for speed-ups')
# classification metrics
from sklearn.metrics import (roc_curve, log_loss, roc_auc_score, 
    accuracy_score, precision_score, recall_score)
# clustering metrics
from sklearn.metrics import (adjusted_mutual_info_score, 
    adjusted_rand_score, completeness_score, 
    fowlkes_mallows_score, homogeneity_score, mutual_info_score, 
    normalized_mutual_info_score, rand_score, v_measure_score)
# regression metrics
from sklearn.metrics import (explained_variance_score, max_error, 
    mean_absolute_error, mean_squared_error, mean_squared_error, 
    mean_squared_log_error, median_absolute_error, r2_score, 
    mean_poisson_deviance, mean_gamma_deviance,
    mean_absolute_percentage_error, #d2_absolute_error_score, 
#     d2_pinball_score, d2_tweedie_score
)

classifier_metrics = [log_loss, roc_auc_score, accuracy_score, 
    precision_score, recall_score]
regression_metrics = [explained_variance_score, max_error, 
    mean_absolute_error, mean_squared_error, mean_squared_error, 
    mean_squared_log_error, median_absolute_error, r2_score, 
    mean_poisson_deviance, mean_gamma_deviance,
    mean_absolute_percentage_error, #d2_absolute_error_score, 
#     d2_pinball_score, d2_tweedie_score
]
clustering_metrics = [adjusted_mutual_info_score, 
    adjusted_rand_score, completeness_score, 
    fowlkes_mallows_score, homogeneity_score, mutual_info_score, 
    normalized_mutual_info_score, rand_score, v_measure_score]
from sklearn.base import is_classifier, is_regressor
from packaging.version import Version
# these imports are for basic usage with sklearn, others
# may be needed for tensorflow, etc.
from sklearn import __version__ as sklearnver
if Version(sklearnver) < Version("0.23.0"):
    from sklearn.externals import joblib
else:
    import joblib

#=====custom imports
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from azureml.core import Model
import os
import numpy as np
import mlflow
from sklearn import __version__ as sklearnver
from packaging.version import Version
if Version(sklearnver) < Version("0.23.0"):
    from sklearn.externals import joblib
else:
    import joblib

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [14]:
# can be local files or pip/conda installed
# be sure to mark in the appropriate section 
# of the .yml file
from example_imports.extra import test_func

#=====base setup
# including this so that linting works and does not complain about missing variables
# output_path = './outputs'
# os.makedirs(output_path, exist_ok=True)

# run = Run.get_context()
# workspace = run.experiment.workspace
# env = run.get_environment()


#=====set experiment and model name
experiment_name = 'default_experiment_name'
model_name = 'default_model_name'

#===== start the run
start = time()
metrics = {}
tags, desc, model_framework = None, None, None
# model_path = os.path.join(output_path,'model.pkl')

#=====import data
X, y = load_diabetes(return_X_y=True)


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)
data = {"X_train":  X_train, "y_train": y_train,
        "X_test": X_test, "y_test": y_test}

#=====custom training code
# list of numbers from 0.0 to 1.0 with a 0.05 interval
alphas = np.arange(0.0, 1.0, 0.05)

best_run_model_filename, model = None, None
best_mse = np.inf
for alpha in alphas:
    # Use Ridge algorithm to create a regression model
    reg = Ridge(alpha=alpha)
    reg.fit(X_train, y_train)

    preds = reg.predict(X_test)
    mse = mean_squared_error(preds, y_test)
#     run.log('alpha', alpha)
#     run.log('mse', mse)

#     model_file_name = os.path.join(output_path, 
#         'ridge_{0:.2f}.pkl'.format(alpha))
    # save model in the outputs folder so it automatically get uploaded
#     with open(model_file_name, "wb") as file:
#         joblib.dump(value=reg, filename=file)
    if best_run_model_filename is None or mse < best_mse:
#         best_run_model_filename = model_file_name
        best_mse = mse
        model = reg
        
    print('alpha is {0:.2f}, and mse is {1:0.2f}'.format(alpha, mse))
    


#=====save the model
# this may need to be updated based on the type of model 
# implemented: i.e., sklearn vs tensorflow, etc.
# with open(model_path, "wb") as file:
#     joblib.dump(value=model, filename=file)

alpha is 0.00, and mse is 3424.32
alpha is 0.05, and mse is 3408.92
alpha is 0.10, and mse is 3372.65
alpha is 0.15, and mse is 3345.15
alpha is 0.20, and mse is 3325.29
alpha is 0.25, and mse is 3311.56
alpha is 0.30, and mse is 3302.67
alpha is 0.35, and mse is 3297.66
alpha is 0.40, and mse is 3295.74
alpha is 0.45, and mse is 3296.32
alpha is 0.50, and mse is 3298.91
alpha is 0.55, and mse is 3303.14
alpha is 0.60, and mse is 3308.70
alpha is 0.65, and mse is 3315.36
alpha is 0.70, and mse is 3322.90
alpha is 0.75, and mse is 3331.17
alpha is 0.80, and mse is 3340.02
alpha is 0.85, and mse is 3349.36
alpha is 0.90, and mse is 3359.09
alpha is 0.95, and mse is 3369.13


In [15]:
#=====standard metrics
metrics["training_time"] = time() - start
predictions = model.predict(data['X_train'])
eg_model_key = None
try:
    if is_classifier(model):
        metrics_functions = classifier_metrics
    elif is_regressor(model):
        metrics_functions = regression_metrics
    else:
        metrics_functions = clustering_metrics
    if isinstance(predictions,dict):
        for mkey, pred in predictions.items():
            metrics.update({f'{mkey}.train_{f.__name__}': f(data['y_train'][mkey], pred) 
                    for f in metrics_functions})
        if isinstance(data['X_train'],dict):
            eg_model_key = mkey
    else:
        metrics.update({f'train_{f.__name__}': f(data['y_train'], predictions) 
                for f in metrics_functions})

    if (data['y_test'][eg_model_key].size > 0 
        if eg_model_key is not None else data['y_test'].size > 0):
        # Predict the transformed test fields
        test_predictions = model.predict(data['X_test'])
        if isinstance(predictions,dict):
            for mkey, pred in test_predictions.items():
                metrics.update({f'{mkey}.test_{f.__name__}': f(data['y_test'][mkey], pred) 
                        for f in metrics_functions})
        else:
            metrics.update({f'test_{f.__name__}': f(data['y_test'], test_predictions) 
                for f in metrics_functions})
except Exception as e:
    warn(f'error trying to evaluate metrics: {str(e)}')

#=====record with MLFlow
# mlflow.start_run(run.id)

#=====save metrics
# if len(metrics):
#     # mlflow.log_metrics({k: v for k,v in metrics.items()})
#     run.log({k: v for k,v in metrics.items()})

#=====register the model (if selected in AzureRun settings)

# # if running on a `compute instance` then MLflow 
# # must be used to save the model
# mlflow.sklearn.log_model(
#     sk_model=model,
#     artifact_path=model_path,
#     registered_model_name=model_name,
#     conda_env=env.python.conda_dependencies.as_dict()
# )

# if using a `compute cluster`
# Model.register(model_name=model_name, 
#     model_path=model_path, workspace=workspace,
#     model_framework=model_framework,
#     description=desc, tags=tags)


#=====save performance plots (if selected in AzureRun settings)
# relies on 'data' dict variable being defined in the custom training 
# code above with the DataFrame or ndarray elements key by 'X_train', 
# 'y_train', etc.
if is_classifier(model):
    roc_curves = {}
    for lbl, Xy in [('train',(data['X_train'],data['y_train'])), ('test',(data['X_test'],data['y_test']))]:
        if ((eg_model_key is None and Xy[0].size > 0) or 
                (eg_model_key is not None and Xy[0][eg_model_key].size > 0)):
            pred_prob = model.predict_proba(Xy[0])
            # num_amb = ((pred_prob < AMBIGUOUS_PROB_RANGE[-1]) & 
            #     (pred_prob > AMBIGUOUS_PROB_RANGE[0])).sum()
            # metrics[lbl + '_numAmb'] = num_amb
            # if debug:
            #     print(f'Number of ambiguous classifications: {lbl:4s} -> {num_amb}')
            if isinstance(pred_prob,dict):
                for mkey,pp in pred_prob.items():
                    roc_curves['.'.join([mkey,lbl])] = roc_curve(Xy[1][mkey], pred_prob[mkey])
            else:
                pred_prob = pred_prob[:,1]
                roc_curves[lbl] = roc_curve(Xy[1], pred_prob)


    fig, ax = plt.subplots()
    annot_pos = []
    for lbl, roc_data in roc_curves.items():
        ax.plot(*roc_data[0][:2],linewidth=2,
                label=f'{lbl.capitalize()} | AUC={metrics[lbl+"_roc_auc_score"]:.2f}')
        thr = roc_data[0][2]
        subselect = np.arange(len(thr))[
            np.array([(len(thr)-1)*posfrac for posfrac in [0.25,0.55,0.8]],dtype=int)]
        for thres, x, y in zip(thr[subselect],
                                roc_data[0][0][subselect],
                                roc_data[0][1][subselect]):
            pos = np.array([x*0.9,y*1.1])
            if any([np.linalg.norm(pos-prev) < 0.05 for prev in annot_pos]):
                continue
            annot_pos.append(pos)
            ax.annotate(f'thres:{thres:.2f}',pos,fontsize=10,alpha=0.8)
    ax.plot([0,1],[0,1],color='tab:red',linestyle='--',linewidth=2,label='Random')
    ax.plot([0,0,1],[0,1,1],color='k',linestyle=':',linewidth=2,label='Ideal')
    ax.set_title('ROC Curve (missed events vs. false alarms trade-off)',fontsize=14)
    ax.set_xlabel('False positive rate',fontsize=14)
    ax.set_ylabel('True positive rate',fontsize=14)
    ax.legend(fontsize=13)
    # output_plots = './outputs/plots'
    # os.makedirs(output_plots, exist_ok=True)
    plt.show()
#=====closing

#=====DONE

In [16]:
metrics

{'training_time': 1.8200798034667969,
 'train_explained_variance_score': 0.46948727708253357,
 'train_max_error': 137.03389400532245,
 'train_mean_absolute_error': 48.14172392491674,
 'train_mean_squared_error': 3252.4130974317027,
 'train_mean_squared_log_error': 0.19893750002393598,
 'train_median_absolute_error': 45.60558677721997,
 'train_r2_score': 0.4694872770825337,
 'train_mean_poisson_deviance': 22.165331642285935,
 'train_mean_gamma_deviance': 0.17424779754459263,
 'train_mean_absolute_percentage_error': 0.4516992252144681,
 'test_explained_variance_score': 0.34313823103258556,
 'test_max_error': 158.6296152731937,
 'test_mean_absolute_error': 46.436906339230525,
 'test_mean_squared_error': 3369.134739913048,
 'test_mean_squared_log_error': 0.1707247309952196,
 'test_median_absolute_error': 41.90404053031568,
 'test_r2_score': 0.3429830961738376,
 'test_mean_poisson_deviance': 21.62696947106484,
 'test_mean_gamma_deviance': 0.15615695215043565,
 'test_mean_absolute_percentage